In [30]:
from Models.groq import llm
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from langchain_core.runnables import RunnableLambda,RunnableBranch
from typing import Literal

In [31]:
class validity_output(BaseModel):
    question:str
    type:Literal["valid","ambiguous","invalid"]
validity_parser=PydanticOutputParser(pydantic_object=validity_output)
validity_prompt = PromptTemplate(
    input_variables=["question"],
    partial_variables={
        "format_instructions":validity_parser.get_format_instructions()
    },
    template="""

You are an expert on general topics.

Given the question:
{question}

- valid: factual, time-independent, and semantically correct
- ambiguous: factual but time-dependent or missing context
- invalid: semantically impossible or uses incorrect roles

Return as per below instruction

{format_instructions}
"""
)
validity_chain = prompt=validity_prompt | llm | validity_parser


In [32]:
class rewrite_output(BaseModel):
    question:str
rewrite_parser=PydanticOutputParser(pydantic_object=rewrite_output)
rewrite_prompt = PromptTemplate(
    input_variables=["question"],
    partial_variables={
        "format_instructions": rewrite_parser.get_format_instructions()
    },
    template="""
You are a query rewriter.

Task:
- Rewrite the given question to be clear and unambiguous
- Preserve the original intent
- DO NOT answer the question
- DO NOT define or explain concepts
- DO NOT create schemas or metadata

Input question:
{question}

Output rules:
- Return ONLY the rewritten question
- Follow the JSON format exactly
- No extra text

{format_instructions}
"""
)

rewrite_chain=rewrite_prompt|llm|rewrite_parser

In [33]:
class intent_output(BaseModel):
    question:str
    intent: Literal["question", "definition", "comparison", "opinion"]
intent_parser=PydanticOutputParser(pydantic_object=intent_output)
intent_prompt=PromptTemplate(input_variables=["question"],template="""
classify this {question} based on these categories question,definition,comparison,opinion
Choose the PRIMARY intent only.
Ignore secondary requests.
Return exactly one intent.
return according to below format
{format_instructions}
""",
partial_variables={"format_instructions":intent_parser.get_format_instructions()}
)
intent_chain=intent_prompt|llm|intent_parser


In [34]:
class answer_output(BaseModel):
    question:str
    answer:str
answer_parser=PydanticOutputParser(pydantic_object=answer_output)
answer_prompt=PromptTemplate(input_variables=["question","type"],template="""
Answer for this question {question} which is of this type {type} and return in the below format
{format_instructions}
""",
partial_variables={"format_instructions":answer_parser.get_format_instructions()})
answer_chain=answer_prompt|llm|answer_parser

In [35]:
class summary_output(BaseModel):
    question:str
    summary:str

summary_parser=PydanticOutputParser(pydantic_object=summary_output)

summary_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    partial_variables={
        "format_instructions": summary_parser.get_format_instructions()
    },
    template="""
You are a JSON generator.

Summarize the following question and answer.

Question:
{question}

Answer:
{answer}

Rules:
- Output ONLY valid JSON
- No explanations
- No extra text

{format_instructions}
"""
)


summary_chain=summary_prompt|llm|summary_parser

In [36]:
pipeline=(
    validity_chain
    | RunnableBranch(
        (
        lambda x:x.type=="valid",
        RunnableLambda(lambda x:{"question":x.question})
        |rewrite_chain
        |RunnableLambda(lambda x:{"question":x.question})
        |intent_chain
        |RunnableBranch(
        (
        lambda x:x.intent!="opinion",
        RunnableLambda(lambda x:{"question":x.question,"type":x.intent})
        |answer_chain
        |RunnableLambda(lambda x:{"question":x.question,"answer":x.answer})
        |summary_chain  
        |RunnableLambda(lambda x:x.summary)
        ),
        (
            RunnableLambda(lambda _:"sorry i cant answer to this")
        )
        ),
        )
        ,
        (lambda x:x.type=='ambiguous',
        RunnableLambda(lambda x:{"question":"As of today"+x.question})
        |rewrite_chain
        |RunnableLambda(lambda x:{"question":x.question})
        |intent_chain
        |RunnableBranch(
            (
            lambda x:x.intent!="opinion",
            RunnableLambda(lambda x:{"question":x.question,"type":x.intent})
            |answer_chain
            |RunnableLambda(lambda x:{"question":x.question,"answer":x.answer})
            |summary_chain  
            |RunnableLambda(lambda x:x.summary)
            ),
            (
                RunnableLambda(lambda _:"sorry i cant answer to this")
            )
        ),
        ),
        RunnableLambda(lambda _:"this is not valid,it will not be proceeded further")
    )
)

In [ ]:
while True:
    query=input("Enter your query: ").lower()
    if query=="exit":
        break
    result=pipeline.invoke({"question":query})
    print(result)

Argentina
A perceptron is a type of feedforward neural network that uses a single layer of weights and biases to classify inputs into one of two categories.
this is not valid,it will not be proceeded further
History, Science, Technology, Literature, Geography, Health and Medicine, Entertainment, Business and Economics, Education
Tungsten is the hardest element found on our planet, with a Mohs hardness of 8-9 and a Vickers hardness of 2000-3000.
this is not valid,it will not be proceeded further
this is not valid,it will not be proceeded further
It's difficult to determine the world's best programmer as it's subjective and can vary depending on the criteria used to measure programming skills.
Codeforces problems, algorithms, and data structures such as graph theory, dynamic programming, and greedy algorithms.
Konstantin Chernikov (also known as pavelkun) is currently considered one of the best programmers based on Codeforces and other relevant sources.
History, Science, Technology, Lite